In [6]:
# pip install -e "git+https://github.com/openskynetwork/opensky-api.git#egg=opensky-api&subdirectory=python"

ERROR: Could not find a version that satisfies the requirement threading (from versions: none)
ERROR: No matching distribution found for threading

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from opensky_api import OpenSkyApi
import pandas as pd
import threading

api = OpenSkyApi(username="tobytcc", password="RootRoot")

In [2]:
df = pd.read_csv("training_data_v1.csv")
icao_list = df["icao24"].to_list()
icao_list_1 = icao_list[0:1923]
icao_list_2 = icao_list[1924:3847]
icao_list_3 = icao_list[3848:5772]
icao_list_4 = icao_list[5773:7696]
icao_list_5 = icao_list[7697:9621]

success_list_1 = []
success_list_2 = []
success_list_3 = []
success_list_4 = []
success_list_5 = []

stored_list = []

In [3]:
def get_aircraft_info(icao_list, count, success_list):
    successcount = 0
    for icao in icao_list:
        count += 1
        print(f'({count}) -- {icao}')
        try:
            data = api.get_flights_by_aircraft(icao, 1697345171-86400, 1697345228+86400)
        except Exception:
            continue

        if data is not None:
            firstSeen = data[0].firstSeen       
            estDepartureAirport = data[0].estDepartureAirport
            lastSeen = data[0].lastSeen
            estArrivalAirport = data[0].estArrivalAirport
            callsign = data[0].callsign
            successcount += 1
            print(f'Success: {count}{icao} -- {successcount}')

            temp = [icao, firstSeen, estDepartureAirport, lastSeen, estArrivalAirport, callsign]
            success_list.append(temp)


In [4]:
t1 = threading.Thread(target=get_aircraft_info, args=(icao_list_1, 0, success_list_1))
t2 = threading.Thread(target=get_aircraft_info, args=(icao_list_2, 1924, success_list_2))
t3 = threading.Thread(target=get_aircraft_info, args=(icao_list_3, 3848, success_list_3))
t4 = threading.Thread(target=get_aircraft_info, args=(icao_list_4, 5773, success_list_4))
t5 = threading.Thread(target=get_aircraft_info, args=(icao_list_5, 7697, success_list_5))

t1.start()
t2.start()
t3.start()
t4.start()
t5.start()

t1.join()
t2.join()
t3.join()
t4.join()
t5.join()

print("done!")

(1) -- a2e5ec
(1925) -- aa630b
(3849) -- 0d0d41
(5774) -- c055d2
(7698) -- a0cb39
Success: 7698a0cb39 -- 1
(7699) -- acc1eb
Success: 5774c055d2 -- 1
(5775) -- ad14aa
Success: 7699acc1eb -- 2
(7700) -- a03ef7
Success: 7700a03ef7 -- 3
(7701) -- addadb
(1926) -- c078bf(3850) -- a64ab2

(2) -- a678ef
(5776) -- a6ac43
(1927) -- ad24a8
Success: 3850a64ab2 -- 1
(3851) -- a5810c
Success: 5776a6ac43 -- 2
(5777) -- aaf127
Success: 1927ad24a8 -- 1
(1928) -- c05ee1
Success: 5777aaf127 -- 3
(5778) -- 39cf06
(7702) -- a4b50f
Success: 577839cf06 -- 4
(5779) -- a42f8b
Success: 7702a4b50f -- 4
(7703) -- a4c849
Success: 5779a42f8b -- 5
(5780) -- c055f5
Success: 7703a4c849 -- 5
(7704) -- a87c4f
Success: 5780c055f5 -- 6
(5781) -- aca760
Success: 5781aca760 -- 7
(5782) -- c055f2
Success: 5782c055f2 -- 8
(5783) -- 3c4593
(3) -- ac4963
(3852) -- a437c0
Success: 3ac4963 -- 1
(4) -- c047e5
(1929) -- ac04ae
Success: 3852a437c0 -- 2
(3853) -- ab62ad
Success: 4c047e5 -- 2
(5) -- a808cf
Success: 1929ac04ae -- 2
(1

In [5]:
stored_list = success_list_1 + success_list_2 + success_list_3 + success_list_4 + success_list_5
print(len(stored_list))

3820


In [6]:
airline_info = pd.DataFrame(stored_list)
airline_info.to_csv("airline_info.csv")